In [137]:
import requests, pymongo, json, time
from datetime import datetime
from dateutil.relativedelta import relativedelta
from bs4 import BeautifulSoup

In [220]:
DataCollection=[]
MetaData=[]

In [214]:


def call_api(url, payload, p=0):
    # Add the 'page' parameter to the payload.
    payload['page'] = p

    # Get the requested url. Error handling for bad requests should be done in
    # the calling function.
    return requests.get(url, params=payload)


def get_response(r):
    # Use json.loads to read the response text
    raw = json.loads(r.text)

    # Return the meta (hits, etc.) and docs (containing urls'n'stuff) back
    print(type(raw))
    return raw['response']['meta'], raw['response']['docs']


def remove_previously_scraped(coll, docs):
    # Check to see if the mongo collection already contains the docs returned
    # from NYT. Return back a list of the ones that aren't in the collection to
    # be scraped.
    new_docs = []
    for doc in docs:
        # Check fo the document id in mongo. If it finds none, append to
        # new_docs
        cursor = articles.find({'_id': doc['_id']}).limit(1)
        if not cursor.count() > 0:
            new_docs.append(doc)

    if new_docs == []:
        return None

    return new_docs


def scrape_articles():
    page = 0
    while page <= 100:
        print ('Page:', page)

        # Request all of the newest articles matching the search term
        payload  = {'sort': 'newest',
                    'end_date': '20170101',
                    'api-key': API_KEY,
                    'begin_date':'20151001',
                    'news_desk':'Politics'
                   }

        # Call the API with BaseURL + params and page
        r = call_api(NYT_URL, payload, page)

        # Increment the page before we encounter any potential errors
        page += 1

        # Check to see if the metadata didn't come back. USUALLY happens if
        # page > 100. When it does, reset the whole thing, roll the date back
        # one day, sleep for a couple seconds, then keep going.
        if r.status_code != 200:
            page = 0
            print (r.status_code )
            time.sleep(2)
            continue
            
        # Get the meta data & documents from the request
        meta, docs = get_response(r)
        DataCollection.append(docs)
        MetaData.append(meta)


In [215]:
NYT_URL = 'https://api.nytimes.com/svc/search/v2/articlesearch.json?q=election'
api_key_path = "/Users/rohanbansal/Desktop/NY_times_api/api_key.txt"


In [216]:
with open(api_key_path, 'r') as handle:
    API_KEY = handle.read()

In [219]:
scrape_articles()


Page: 0
<class 'dict'>
Page: 1
<class 'dict'>
Page: 2
<class 'dict'>
Page: 3
500
Page: 0
<class 'dict'>
Page: 1
<class 'dict'>
Page: 2
500
Page: 0
<class 'dict'>
Page: 1


KeyboardInterrupt: 

In [172]:
(DataCollection[1][2])

{'abstract': '‘Step by step, Beijing is going back to the old path of dividing, coercing and even threatening and intimidating Taiwan,’ President Tsai Ing-wen said.',
 'web_url': 'https://www.nytimes.com/2016/12/31/world/asia/taiwan-china-tsai-ing-wen-trump.html',
 'snippet': '‘Step by step, Beijing is going back to the old path of dividing, coercing and even threatening and intimidating Taiwan,’ President Tsai Ing-wen said.',
 'lead_paragraph': 'BEIJING — President Tsai Ing-wen of Taiwan sharply criticized China’s leaders on Saturday, saying they had resorted to military and economic threats in order to intimidate the island.',
 'print_section': 'A',
 'print_page': '4',
 'source': 'The New York Times',
 'multimedia': [{'rank': 0,
   'subtype': 'square320',
   'caption': None,
   'credit': None,
   'type': 'image',
   'url': 'images/2017/01/01/world/01TAIWAN/01TAIWAN-square320.jpg',
   'height': 320,
   'width': 320,
   'legacy': {},
   'subType': 'square320',
   'crop_name': 'square32

In [173]:
import pandas as pd

In [196]:
tempFrame = pd.DataFrame.from_dict((DataCollection[2]))
tempFrame

,_id,abstract,byline,document_type,headline,keywords,lead_paragraph,multimedia,news_desk,print_page,print_section,pub_date,section_name,snippet,source,subsection_name,type_of_material,uri,web_url,word_count
0,nyt://article/ac3e7af4-c8d2-503c-9107-d32e3d10...,"With three weeks left in office, the president...","{'original': 'By Michael D. Shear', 'person': ...",article,{'main': 'Obama’s Last Days: Aiding Trump Tran...,"[{'name': 'subject', 'value': 'United States P...","WASHINGTON — Only two days after the election,...","[{'rank': 0, 'subtype': 'square320', 'caption'...",National,1,A,2016-12-31T17:08:33+0000,U.S.,"With three weeks left in office, the president...",The New York Times,Politics,News,nyt://article/ac3e7af4-c8d2-503c-9107-d32e3d10...,https://www.nytimes.com/2016/12/31/us/politics...,1375
1,nyt://article/389b6079-68ac-5fdc-aa9b-2721892a...,"Mrs. Dukakis, a former first lady of Massachus...","{'original': 'By Katharine Q. Seelye', 'person...",article,"{'main': 'Kitty Dukakis, a Beneficiary of Elec...","[{'name': 'persons', 'value': 'Dukakis, Kitty'...","BROOKLINE, Mass. — When Michael Dukakis lost t...","[{'rank': 0, 'subtype': 'square320', 'caption'...",National,13,A,2016-12-31T16:51:05+0000,U.S.,"Mrs. Dukakis, a former first lady of Massachus...",The New York Times,NaN,News,nyt://article/389b6079-68ac-5fdc-aa9b-2721892a...,https://www.nytimes.com/2016/12/31/us/kitty-du...,1684
2,nyt://article/ae20317a-f14e-5b89-ad82-1ee5da14...,President-elect Donald J. Trump has made deals...,{'original': 'By Richard C. Paddock and Eric L...,article,"{'main': 'Trump’s Indonesia Projects, Still Mo...","[{'name': 'persons', 'value': 'Trump, Donald J...","JAKARTA, Indonesia — One resort, planned as th...","[{'rank': 0, 'subtype': 'square320', 'caption'...",Foreign,1,A,2016-12-31T16:09:59+0000,World,President-elect Donald J. Trump has made deals...,The New York Times,Asia Pacific,News,nyt://article/ae20317a-f14e-5b89-ad82-1ee5da14...,https://www.nytimes.com/2016/12/31/world/asia/...,1879
3,nyt://article/8d61b0e9-e69f-5dc5-b584-b4fc37e2...,"In a book about her enigmatic father, Susan Fa...","{'original': 'By Peter Haldeman', 'person': [{...",article,"{'main': 'My Father, the Shapeshifter', 'kicke...","[{'name': 'subject', 'value': 'Children and Ch...",LOS ANGELES — On a warm September evening in t...,"[{'rank': 0, 'subtype': 'square320', 'caption'...",Styles,1,ST,2016-12-31T11:00:25+0000,Fashion & Style,"In a book about her enigmatic father, Susan Fa...",The New York Times,NaN,News,nyt://article/8d61b0e9-e69f-5dc5-b584-b4fc37e2...,https://www.nytimes.com/2016/12/31/fashion/sus...,2333
4,nyt://article/9f051402-f195-5d37-b035-72abf8b4...,It’s time to say goodbye to 2016. Here is how ...,"{'original': 'By Joshua Barone', 'person': [{'...",article,{'main': 'What’s on TV Saturday: New Year’s Ev...,"[{'name': 'subject', 'value': 'Television', 'r...","It’s time to say goodbye to 2016. Somehow, we ...","[{'rank': 0, 'subtype': 'square320', 'caption'...",Culture,8,C,2016-12-31T06:00:19+0000,Arts,It’s time to say goodbye to 2016. Here is how ...,The New York Times,Television,Schedule,nyt://article/9f051402-f195-5d37-b035-72abf8b4...,https://www.nytimes.com/2016/12/31/arts/televi...,501
5,nyt://article/1705a4d0-0b71-578c-8b70-d3f85269...,"The vehicles’ occupants, who appeared to be mo...","{'original': 'By Joseph Goldstein', 'person': ...",article,{'main': 'Diplomatic Vehicles Vacate Russia’s ...,"[{'name': 'persons', 'value': 'Goldstein, Jose...","UPPER BROOKVILLE, N.Y. — Seven miles from here...","[{'rank': 0, 'subtype': 'square320', 'caption'...",Metro,8,A,2016-12-31T02:41:41+0000,New York,"The vehicles’ occupants, who appeared to be mo...",The New York Times,NaN,News,nyt://article/1705a4d0-0b71-578c-8b70-d3f85269...,https://www.nytimes.com/2016/12/30/nyregion/up...,800
6,nyt://article/c8451f38-b68e-5c36-9154-0c07905d...,On leaving 2016 behind.,"{'original': 'By The Editorial Board', 'person...",article,{'main': 'Take a Bad Year. And Make It Be

In [200]:
main_frame = tempFrame
for page in (DataCollection):
    temp_append_frame =pd.DataFrame.from_dict(page)
    main_frame=main_frame.append(temp_append_frame)



(1020, 20)

In [203]:
main_frame.to_csv('News_NYTCollected.csv')

In [40]:
# import csv
# import json
# import requests
# import json as simplejson
# import time
# categories = ['World']

# artsNews = []
# businessNews = []
# sportsNews = []
# obituariesNews = []
# worldNews = []

# def writeTSV(newsList, category):
# 	filename = category + 'News.tsv'
# 	with open(filename, 'w') as the_file:
# 		csv.register_dialect("custom", delimiter=" ", skipinitialspace=True)
# 		writer = csv.writer(the_file, dialect="custom")
# 		for tup in newsList:
# 			writer.writerow(tup)

# def storeNews(newsList, newsJSONContent):
# 	newsList.append([newsJSONContent["web_url"].encode('utf-8'),
# 		newsJSONContent["headline"]["main"].encode('utf-8'),
# 		newsJSONContent["lead_paragraph"].encode('utf-8')])

# for category in categories:
#     num_articles = 0
#     page = 0
#     while num_articles < 2000:
#         URL = 'http://api.nytimes.com/svc/search/v2/articlesearch.json?api-key=xFuEAM2zEE01ved8CLZUa7LoKbsBpDTW&fq=news_desk:("' + category + '")&fl=web_url,headline,news_desk,lead_paragraph,word_count&begin_date=19810601&end_date=20131112&page=' + str(page)
#         print(URL)


#         time.sleep(2)


#         r = requests.get(URL)
#         c = r.content
#         j = simplejson.loads(c)
#         num_articles += len(j["response"]["docs"])
#         page += 1
#         print(num_articles)

#     #		print j["response"]["docs"][9]["news_desk"]
#         for i in range(0,10):
#             if category is 'World':
#                 storeNews(artsNews,j["response"]["docs"][i])
#                 writeTSV(artsNews,"World")


http://api.nytimes.com/svc/search/v2/articlesearch.json?api-key=xFuEAM2zEE01ved8CLZUa7LoKbsBpDTW&fq=news_desk:("World")&fl=web_url,headline,news_desk,lead_paragraph,word_count&begin_date=19810601&end_date=20131112&page=0
10
http://api.nytimes.com/svc/search/v2/articlesearch.json?api-key=xFuEAM2zEE01ved8CLZUa7LoKbsBpDTW&fq=news_desk:("World")&fl=web_url,headline,news_desk,lead_paragraph,word_count&begin_date=19810601&end_date=20131112&page=1
20
http://api.nytimes.com/svc/search/v2/articlesearch.json?api-key=xFuEAM2zEE01ved8CLZUa7LoKbsBpDTW&fq=news_desk:("World")&fl=web_url,headline,news_desk,lead_paragraph,word_count&begin_date=19810601&end_date=20131112&page=2
30
http://api.nytimes.com/svc/search/v2/articlesearch.json?api-key=xFuEAM2zEE01ved8CLZUa7LoKbsBpDTW&fq=news_desk:("World")&fl=web_url,headline,news_desk,lead_paragraph,word_count&begin_date=19810601&end_date=20131112&page=3
40
http://api.nytimes.com/svc/search/v2/articlesearch.json?api-key=xFuEAM2zEE01ved8CLZUa7LoKbsBpDTW&fq=news

KeyError: 'response'